In [6]:

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, TrainerCallback
from peft import LoraConfig, TaskType, get_peft_model
import torch
import psutil
import numpy as np



In [29]:
!pip install trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 7.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.3 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidi

In [7]:
torch.cuda.set_per_process_memory_fraction(1.0, 0)  # Use maximum available memory
torch.cuda.memory_max_split_size_mb = 64  # Set the max split size to avoid fragmentation

In [8]:
def print_memory_footprint():
    # GPU memory usage
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.memory_allocated() / (1024 ** 3)  # Convert to GB
        gpu_memory_cached = torch.cuda.memory_reserved() / (1024 ** 3)  # Cached memory
        print(f"[GPU] Memory Allocated: {gpu_memory:.2f} GB, Cached: {gpu_memory_cached:.2f} GB")
    else:
        print("[GPU] No GPU detected.")

    # CPU memory usage
    memory = psutil.virtual_memory()
    used_memory_gb = memory.used / (1024 ** 3)  # Convert to GB
    total_memory_gb = memory.total / (1024 ** 3)
    print(f"[CPU] Memory Usage: {used_memory_gb:.2f} GB / {total_memory_gb:.2f} GB")

In [9]:
'''
print("First example of blended_skill_talk training set:")
print(dataset['train'][0])
'''

'\nprint("First example of blended_skill_talk training set:")\nprint(dataset[\'train\'][0])\n'

In [10]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token, so we use eos_token

In [11]:
# Load dataset and tokenizer
dataset = load_dataset("allenai/soda")



def tokenize_function(examples):
    # Concatenate dialog turns into a single string for language modeling
    texts = [" ".join(dialog) for dialog in examples["dialogue"]]
    return tokenizer(texts, truncation=True, max_length=512)

# Tokenize datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'])
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(500))

Map:   0%|          | 0/146346 [00:00<?, ? examples/s]

In [12]:
print(dataset)
tokenized_datasets


DatasetDict({
    train: Dataset({
        features: ['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'],
        num_rows: 1191582
    })
    validation: Dataset({
        features: ['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'],
        num_rows: 146346
    })
    test: Dataset({
        features: ['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'],
        num_rows: 148968
    })
})


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1191582
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 146346
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 148968
    })
})

In [13]:
# LoRA configuration for causal language modeling
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=2,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj"],
)

# %%

In [14]:
# Dynamic device assignment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained GPT-2 language model
# Load pre-trained GPT-2 model
#model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(device)
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                                             torch_dtype=torch.bfloat16,).to(device)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Enable gradient checkpointing if you run into memory issues
#model.gradient_checkpointing_enable()

# %%
# Print the model's architecture to inspect the names of the modules
#print(model)

# %%
print_memory_footprint()

Using device: cuda


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

[GPU] Memory Allocated: 2.05 GB, Cached: 2.14 GB
[CPU] Memory Usage: 2.39 GB / 31.35 GB


In [15]:
# %%

# Data collator for language modeling (masks tokens for prediction)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) # mlm=False for causal LM

# Perplexity as metric
#perplexity_metric = evaluate.load("perplexity", module_type="metric")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    shift_logits = torch.tensor(logits[:, :-1, :])
    shift_labels = torch.tensor(labels[:, 1:])
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    ppl = torch.exp(loss).item()
    return {"perplexity": ppl}

# Training args
training_args = TrainingArguments(
    output_dir="./lama-dialog-finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=1000,
    #evaluation_strategy="epoch",
    #save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False,
    bf16=True,  # Enable bfloat16
    fp16=False,  # Disable fp16 to avoid conflicts
)
# %%
print_memory_footprint()

[GPU] Memory Allocated: 2.05 GB, Cached: 2.14 GB
[CPU] Memory Usage: 2.40 GB / 31.35 GB


In [16]:
class MemoryCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        print("\nMemory footprint after evaluation:")
        print_memory_footprint()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    #eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
    callbacks=[MemoryCallback()]
)


/tmp/ipykernel_35/3187942852.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [17]:

print("--- TinyLlama Before Fine-tuning ---")
model_before_finetune = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
             torch_dtype=torch.bfloat16).to(device)

input_text_before = "Hello, how are you?"
input_ids_before = tokenizer.encode(input_text_before, return_tensors="pt").to(device)

print(f"Input: {input_text_before}")
output_before = model_before_finetune.generate(
    input_ids_before,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
generated_response_before = tokenizer.decode(output_before[0], skip_special_tokens=True)
print(f"Generated Response (Before): {generated_response_before}")
print("---")


--- TinyLlama Before Fine-tuning ---


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Input: Hello, how are you?
Generated Response (Before): Hello, how are you?
I am fine, thank you.
How are you? I am fine, thank you.
Can you repeat that again?
Can you repeat that again, please?
Can you repeat that again,
---


In [18]:

print("--- TinyLlama Before Fine-tuning ---")
#model_before_finetune = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

input_text_before = "The USA celebrate independence day on "
input_ids_before = tokenizer.encode(input_text_before, return_tensors="pt").to(device)

print(f"Input: {input_text_before}")
output_before = model_before_finetune.generate(
    input_ids_before,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
generated_response_before = tokenizer.decode(output_before[0], skip_special_tokens=True)
print(f"Generated Response (Before): {generated_response_before}")
print("---")


--- TinyLlama Before Fine-tuning ---
Input: The USA celebrate independence day on 
Generated Response (Before): The USA celebrate independence day on 4th July.

2. The USA celebrate independence day on 4th July.

3. The USA celebrate independence day on 4th July.

4. The
---


In [19]:
any(p.requires_grad for p in model.parameters())

True

In [20]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=500, training_loss=0.75694677734375, metrics={'train_runtime': 554.373, 'train_samples_per_second': 3.608, 'train_steps_per_second': 0.902, 'total_flos': 3150837652340736.0, 'train_loss': 0.75694677734375, 'epoch': 1.0})

In [21]:
trainer.save_model('TinyLlama-new-1000')

In [22]:
print_memory_footprint()

[GPU] Memory Allocated: 4.12 GB, Cached: 9.63 GB
[CPU] Memory Usage: 3.10 GB / 31.35 GB


In [23]:
# Dynamic device assignment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# For generation, we'll load the model and use the `generate` method
model_for_generation = AutoModelForCausalLM.from_pretrained('TinyLlama-new-1000').to(device)
#model_for_generation = get_peft_model(model_for_generation, lora_config) # Ensure LoRA is applied

input_text = "Hello, how are you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")

print_memory_footprint()

Using device: cuda

Generating response:


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Input: Hello, how are you?
Generated Response: Hello, how are you? I'm good, thanks. How about you? I'm doing well. I'm glad to hear that. You sound good too. That's good to hear. I'm glad to hear that too. You're doing great. That's great to hear. I'm glad to hear that too. You're doing great. That's great to hear. I'm glad to hear that too. You're doing great. That's great to hear. I'm glad to hear that too. You're doing great. That's great to hear. I'm glad to hear that too. You're doing great. That's
[GPU] Memory Allocated: 8.22 GB, Cached: 10.63 GB
[CPU] Memory Usage: 4.23 GB / 31.35 GB


In [24]:
input_text = "The USA celebrate independence day on "
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")


Generating response:
Input: The USA celebrate independence day on 
Generated Response: The USA celebrate independence day on 4th July.

2. India Independence Day is celebrated on 15th August.

3. Canada celebrates its 150th birthday on July 1st.

4. Australia celebrates its 238th birthday on 26th January.

5. New Zealand celebrates its 238th birthday on 26th January.

6. South Africa celebrates its 238th birthday on 26th January.

7. United States celebrates its 241st birthday on 4th July.

8. United Kingdom celebrates its


In [25]:
input_text = "So light is made up of colors?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")


Generating response:
Input: So light is made up of colors?
Generated Response: So light is made up of colors?
Yes, light is made up of colors. Light is made up of electromagnetic waves, which are made up of tiny particles called photons. Each photon has a specific wavelength, which determines its color. The colors of light are determined by the wavelength of the photons that make up the light. The colors of light are determined by the frequency of the photons. The higher the frequency, the shorter the wavelength, and the more blue or violet the color. The lower the frequency, the longer the wavelength, and the more green or yellow the color. The colors of light are determined by the angle at which the photons are incident


In [26]:
small_eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 500
})

In [27]:
from nltk.translate.bleu_score import sentence_bleu


# Tokenize the generated response
candidate = generated_response.split()

# Define reference responses (tokenized)
references = [
    "Yes, that's right. Light is made up of different colors, and when those colors mix together, we see white light. But light can also bend and bounce off surfaces.",
    "How does it do that?",
    "When light hits a surface, it can either reflect off of the surface or refract through the surface. Reflection is when the light bounces off of the surface, and refraction is when the light bends as it passes through the surface.",
    "So when light reflects off of a mirror, it's because the light is bouncing off of the surface of the mirror?",
    "That's right. Mirrors are very good at reflecting light because they have a smooth, flat surface.",
    "And when light refracts through something like a prism, it's because the light is bending as it passes through the surface of the prism?",
    "That's right. When light passes through a prism, it bends because the Prism bends the light."
]

# Tokenize each reference
reference_tokens = [ref.split() for ref in references]

# Calculate BLEU scores
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4  # Helps with zero scores for higher n-grams

print('Individual 1-gram: %f' % sentence_bleu(reference_tokens, candidate, weights=(1, 0, 0, 0), smoothing_function=smoothie))
print('Individual 2-gram: %f' % sentence_bleu(reference_tokens, candidate, weights=(0, 1, 0, 0), smoothing_function=smoothie))
print('Individual 3-gram: %f' % sentence_bleu(reference_tokens, candidate, weights=(0, 0, 1, 0), smoothing_function=smoothie))
print('Individual 4-gram: %f' % sentence_bleu(reference_tokens, candidate, weights=(0, 0, 0, 1), smoothing_function=smoothie))


Individual 1-gram: 0.232759
Individual 2-gram: 0.069565
Individual 3-gram: 0.026316
Individual 4-gram: 0.017699


# DPO

In [ ]:

from trl import DPOConfig, DPOTrainer
torch.cuda.empty_cache()
Sft_model = AutoModelForCausalLM.from_pretrained('TinyLlama-new-1000').to(device)
train_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")
train_dataset = train_dataset.select(range(2000))
# print(train_dataset)
# # DPO training config
# training_args = DPOConfig(
#     output_dir="TinyLlama-0.5B-DPO",
#     logging_steps=100,
#     per_device_train_batch_size=1,
#     num_train_epochs=1,
#     remove_unused_columns=False,
#     bf16=True,  # Enable bfloat16
#     fp16=False,  # Disable fp16 to avoid conflicts
# )
# trainer = DPOTrainer(model=Sft_model, args=training_args, processing_class=tokenizer, train_dataset=train_dataset)
# trainer.train()